In [1]:
# from baselines.macbf import *
from arm_env2 import *

pybullet build time: Dec  1 2021 18:33:43


# Evaluate Static

In [2]:
# save the environment,dataset,method,density,collision_per_env,collision_per_agent,success_rate,path_length

def get_reward(num_collides, num_success, path_length, agent_num):
    return (num_collides*(-1) + num_success*10)/agent_num + -1e-2*path_length

def get_safe_rate(num_collides, path_length, agent_num):
    return 1-(num_collides/(path_length*agent_num))

In [3]:
from tqdm import tqdm
import gc
from copy import deepcopy
import torch
import numpy as np
from torch import nn
import math
from models import *   
from core import generate_default_model_name
import pickle as pkl
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

SAVE_GIF = False
ONLY_SHOW_COLLIDE = True
VERBOSE = False

dataset_mode = 'static'
decompose = None

bnn = create_network()
bnn.load_state_dict(torch.load(BMODEL_PATH, map_location=device))
bnn.eval();

import copy

for density in [2,3,4]:
    
    collideds = []
    dones = []
    lengths = []

    paths = []
    rewards = []
    safe_rates = []

    path = f'gifs/0513/{project_name}_{version_name}/{density}'
    os.makedirs(path, exist_ok=True)

    for v_idx in tqdm(range(100)):

        torch.manual_seed(v_idx)
        random.seed(v_idx)
        np.random.seed(v_idx)    

        ENV_CONFIG['randomize'] = False
        if density == 4:
            arm_ids = [0,1,2,3]
        else:
            arm_ids = np.random.choice(4, size=density, replace=False)
        env = create_env(num_agents=density, arm_ids=arm_ids)
        if project_name=='drone':
            env.world.agents[:, 3:6] = 0
        if SAVE_GIF:
            gif_file = f'gifs/0513/{project_name}_{version_name}/{density}/'+str(v_idx)+f'_decompose_lie.gif'
        else:
            gif_file = None
        collided, done, path = infer(env,bnn,verbose=VERBOSE,n_action=10000,seed=v_idx,
                                     spatial_prop=False,lie_derive_safe=False,decompose=decompose,
                                     stop_at_collision=False,need_gif=gif_file)
        num_collides = 0
        num_success = np.array([False]*env.num_agents)
        for timestep in path:
            env.world.agents = timestep
            env.set_config(timestep)
            status = env.world.get_status()
            num_collides += sum(['danger' in s for s in status])
            num_success = num_success | np.array(['done' in s for s in status])
        rewards.append(get_reward(num_collides, num_success.sum(), len(path), env.num_agents))
        safe_rates.append(get_safe_rate(num_collides, len(path), env.num_agents))

        collideds.append(collided)
        dones.append(done)
        lengths.append(len(path))
        paths.append(path)

    print(density, np.any(collideds, axis=-1).mean(), np.mean(collideds), np.mean(dones), np.mean(lengths))

    suffix = '' if decompose=='random' else 'no_decompose_'
    with open(f'results/{project_name}_{version_name}_{suffix}{dataset_mode}_{density}.pkl', 'wb') as f:
        pkl.dump({'collideds': collideds,
                  'dones': dones,
                  'lengths': lengths,
                  'paths': paths}, f)

    print(f'{project_name},{dataset_mode},{version_name},{density},{np.mean(safe_rates)},{np.mean(rewards)},{np.std(safe_rates)},{np.std(rewards)}')    

/home/rainorangelemon/anaconda3/envs/gnn/lib/python3.8/site-packages/torch/nn/modules/lazy.py:175: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '
100%|██████████| 100/100 [00:32<00:00,  3.07it/s]


2 0.19 0.1 1.0 29.5
ur5,static,v10,2,0.9922902054438265,9.47,0.019691871150750977,0.6505474617581717


100%|██████████| 100/100 [00:54<00:00,  1.83it/s]


3 0.92 0.6733333333333333 1.0 33.94
ur5,static,v10,3,0.8965727327656036,6.577266666666668,0.09583596307617764,2.5510556150564634


100%|██████████| 100/100 [00:59<00:00,  1.69it/s]

4 1.0 0.795 0.99 34.9
ur5,static,v10,4,0.7720950343421877,2.6234999999999995,0.04098741769201187,1.3774660612878997
